In [1]:
import pandas as pd
import csv

import os, os.path
import sys

# All chosen fields for survey 1
A grass hay field, "106433_WSDA_SF_2017", was missed from meeting PDF! how the hell did this happen?
- I had dropped grass hay from disagreement table! 

In [2]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choices_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names

chosen_fields_set1 = pd.DataFrame()
for a_choice_sheet in choices_set_1_sheet_names:    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)
    chosen_fields_set1 = pd.concat([chosen_fields_set1, a_choice_sheet])

In [3]:
choices_set_2_xl = pd.ExcelFile(perry_dir + "set_2_handPicked.xlsx")
choices_set_2_sheet_names = choices_set_2_xl.sheet_names  # see all sheet names

chosen_fields_set2 = pd.DataFrame()
for a_choice_sheet in choices_set_2_sheet_names:    
    # read a damn sheet
    a_choice_sheet = choices_set_2_xl.parse(a_choice_sheet)
    chosen_fields_set2 = pd.concat([chosen_fields_set2, a_choice_sheet])

In [4]:
print (len(chosen_fields_set2.ID))
print (len(chosen_fields_set2.ID.unique()))

322
322


In [5]:
# # check if there are overlap between chosen fields in set 1 and set 2
# set1_uniqueIDs = list(chosen_fields_set1.ID.unique())
# set2_uniqueIDs = list(chosen_fields_set2.ID.unique())

# A = chosen_fields_set2[chosen_fields_set2.ID.isin(set1_uniqueIDs)]
# B = chosen_fields_set1[chosen_fields_set1.ID.isin(set2_uniqueIDs)]
# A.shape

# Expert Set 1 - post meeting

In [19]:
import pandas as pd

ML_data_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
expert_set_1_meeting_Consensus = pd.read_csv(ML_data_dir + "expert_set1_postmeeting_consensus.csv")
expert_set_1_meeting_Consensus.head(2)

,ID,Vote
0,99837_WSDA_SF_2017,2
1,114615_WSDA_SF_2017,1


In [20]:
print (expert_set_1_meeting_Consensus.shape)
print (expert_set_1_meeting_Consensus.Vote.unique())

(111, 2)
[2 1]


# Expert Set 1 - pre-meeting

In [21]:
expert_set1_premeeting_consensus = pd.read_csv(ML_data_dir + "expert_set1_premeeting_consensus.csv")
expert_set1_premeeting_consensus.head(2)

,ID,PerrysVote,AndrewsVote,TimsVote,PerryVCorrected,AndrewVCorrected,TimsVCorrected,Vote
0,157415_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,1
1,155601_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,1


In [23]:
drop_cols = ['PerrysVote', 'AndrewsVote', 'TimsVote',
             "PerryVCorrected", "AndrewVCorrected", "TimsVCorrected"]
expert_set1_premeeting_consensus.drop(labels=drop_cols, axis=1, inplace=True)

# Concatenate set 1 pre and post meeting labels

In [26]:
expert_set_1_labels = pd.concat([expert_set_1_meeting_Consensus, 
                                 expert_set1_premeeting_consensus[['ID', 'Vote']]])

# we need this shit, cuz some fields were requested to be discussed or had comment
expert_set_1_labels.drop_duplicates(inplace=True) 
expert_set_1_labels.shape

(285, 2)

# Read Experts' Set 2 pre-meeting consensus

In [27]:
expert_set2_premeeting_consensus = pd.read_csv(ML_data_dir + "expert_set2_premeeting_consensus.csv")
expert_set2_premeeting_consensus.head(2)

,ID,PerryV,AndrewV,PerryVCorrected,AndrewVCorrected,Vote
0,160820_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,1
1,161383_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,1


In [31]:
set1_IDs = list(set_1_labels.ID.unique())

#
#    Get rid of repeated fields
#
expert_set2_premeeting_consensus=expert_set2_premeeting_consensus[~expert_set2_premeeting_consensus\
                                                                  .ID.isin(set1_IDs)]

# Non-Expert Labels Set-2

### Pre-Meeting Consensus

In [76]:
NE_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/nonExpert_set2_fields/"
NE_S2_PreMeeting=pd.read_csv(NE_dir+"NE_set2_premeeting_Consensus_IDs_Votes_IncludesUnsure.csv")


# Drop the Unsure ones. We will read it in the post meeting part.
NE_S2_PreMeeting=NE_S2_PreMeeting[NE_S2_PreMeeting.HosseinV!="Unsure"]

# NE_S2_PreMeeting.rename(columns = {'HosseinV':'Vote'}, inplace = True)

NE_S2_PreMeeting["Vote"]=0
NE_S2_PreMeeting.loc[NE_S2_PreMeeting.HosseinV=="Single", "Vote"]=1
NE_S2_PreMeeting.loc[NE_S2_PreMeeting.HosseinV=="Double", "Vote"]=2

NE_S2_PreMeeting=NE_S2_PreMeeting[["ID", "Vote"]]

In [77]:
# Post-Meeting Consensus

# Concatenate DataFrames

In [78]:
train_labels = pd.concat([expert_set_1_labels, 
                          expert_set2_premeeting_consensus[['ID', 'Vote']],
                          NE_S2_PreMeeting])
train_labels.reset_index(drop=True, inplace=True)
# train_labels.drop_duplicates(inplace=True)
train_labels.shape

(1753, 2)

In [79]:
out_folder = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
out_name = out_folder + "train_labels.csv"
train_labels.to_csv(out_name, index = False)